# ReAct: Reasoning + Acting

Building on:
- **001-003**: Basic Branch usage and function calling
- **004**: Conversation patterns and context

**This notebook**: Introduction to ReAct - getting AI to think step-by-step and use tools strategically.

In [1]:
from lionagi import Branch, iModel
import json
from IPython.display import Markdown

## What is ReAct?

**ReAct** = **Reasoning** + **Acting**

Instead of just giving an answer, the AI:
1. **Thinks** about the problem (reasoning)
2. **Uses tools** when needed (acting)
3. **Repeats** until the problem is solved

This creates more reliable, step-by-step problem solving.

## Simple Example: Math Problem Solver

Let's start with a basic calculator that shows its reasoning:

In [2]:
# Simple calculator functions
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float) -> float:
    """Divide two numbers."""
    if b == 0:
        return "Error: Cannot divide by zero"
    return a / b


# Create a ReAct-enabled branch
calculator = Branch(
    system="You solve math problems step by step. Think about what you need to calculate, then use tools.",
    tools=[add, multiply, divide],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

print("Calculator ready with ReAct capabilities!")

Calculator ready with ReAct capabilities!


## Using ReAct for Problem Solving

In [3]:
# Problem: Calculate the total cost of 3 items at $15.50 each, plus 8% tax
problem = (
    "I bought 3 items at $15.50 each. What's the total cost including 8% tax?"
)

# Use ReAct to solve step-by-step
solution = await calculator.ReAct(
    instruct={"instruction": problem},
    max_extensions=5,  # Limit the number of reasoning steps
    verbose=True,  # Show the reasoning process
)

print("\n🎯 Final Answer:")
print(solution)

Invoking action multiply with {'a': 3, 'b': 15.5}.
Action multiply invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: First, calculate the total cost of 3 items before tax by multiplying 3 by 15.50.
planned_actions:
  - action_type: multiply
    description: Calculate the total price for 3 items at $15.50 each.
extension_needed: True
milestone: Calculate total price before tax.
action_strategy: sequential
action_batch_size:

reason:
  title: Step 1: Calculate total before tax
  content: We need the total cost of the 3 items to then calculate the tax amount.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: multiply
    arguments:
      a: 3
      b: 15.5
action_responses:
  - function: multiply
    arguments:
      a: 3
      b: 15.5
    output: 46.5
```
---------

Invoking action multiply with {'a': 46.5, 'b': 0.08}.
Action multiply invoked, status: completed.


### ReAct Round No.2 Analysis:
```yaml
analysis: We have the total price before tax as $46.5. Now, calculate 8% tax on this amount to determine the tax value.
planned_actions:
  - action_type: multiply
    description: Calculate the tax amount by multiplying the total price before tax by 8%.
extension_needed: True
milestone: Calculate tax amount at 8%.
action_strategy: sequential
action_batch_size:

reason:
  title: Step 2: Calculate tax amount
  content: Calculating tax (8%) on the total price before tax is essential to find the total cost including tax.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: multiply
    arguments:
      a: 46.5
      b: 0.08
action_responses:
  - function: multiply
    arguments:
      a: 46.5
      b: 0.08
    output: 3.72
```
---------

Invoking action add with {'a': 46.5, 'b': 3.72}.
Action add invoked, status: completed.


### ReAct Round No.3 Analysis:
```yaml
analysis: The total cost before tax is $46.5 and the tax amount is $3.72. Adding these will give the total cost including tax.
planned_actions:
  - action_type: add
    description: Add the total price before tax and the tax amount to get the total cost including tax.
extension_needed: False
milestone: Calculate total cost including tax.
action_strategy: sequential
action_batch_size:

reason:
  title: Step 3: Calculate total cost including tax
  content: Adding the tax to the original total gives the final amount to be paid.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: add
    arguments:
      a: 46.5
      b: 3.72
action_responses:
  - function: add
    arguments:
      a: 46.5
      b: 3.72
    output: 50.22
```
---------

### ReAct Final Answer:
The total cost for 3 items at $15.50 each including 8% tax is $50.22.
---------


🎯 Final Answer:
The total cost for 3 items at $15.50 each including 8% tax is $50.22.


## Understanding the ReAct Process

Let's look at what happened in the conversation:

In [4]:
# View the reasoning steps
conversation = calculator.to_df()
print(f"Total conversation steps: {len(conversation)}")
print("\nStep-by-step breakdown:")

for i, row in conversation.iterrows():
    if row["role"] == "user":
        print(f"\n👤 User: {str(row['content'])[:100]}...")
    elif row["role"] == "assistant":
        print(f"🤖 Assistant: {str(row['content'])[:100]}...")
    elif row["role"] == "action":
        print(f"🔧 Tool Used: {str(row['content'])[:100]}...")

Total conversation steps: 15

Step-by-step breakdown:

👤 User: {'context': [], 'instruction': "I bought 3 items at $15.50 each. What's the total cost including 8% ...
🤖 Assistant: {'assistant_response': '```json\n{"planned_actions":[{"action_type":"multiply","description":"Calcul...
🔧 Tool Used: {'action_request': {'function': 'multiply', 'arguments': {'a': 3, 'b': 15.5}}, 'action_response_id':...
🔧 Tool Used: {'action_request_id': '1b01754d-71f8-4e34-9c73-62772daf98cc', 'action_response': {'function': 'multi...

👤 User: {'context': [{'action_request_id': '1b01754d-71f8-4e34-9c73-62772daf98cc', 'action_response': {'func...
🤖 Assistant: {'assistant_response': '```json\n{"planned_actions":[{"action_type":"multiply","description":"Calcul...
🔧 Tool Used: {'action_request': {'function': 'multiply', 'arguments': {'a': 46.5, 'b': 0.08}}, 'action_response_i...
🔧 Tool Used: {'action_request_id': 'fb05d581-b62e-47b9-b6f1-d3aa7022d525', 'action_response': {'function': 'multi...

👤 User: {'context

## Practical Example: Research Assistant

Let's build a simple research assistant that can look up information:

In [5]:
# Simple "database" functions
company_data = {
    "apple": {"founded": 1976, "employees": 164000, "revenue": "$394.3B"},
    "google": {"founded": 1998, "employees": 182000, "revenue": "$307.4B"},
    "microsoft": {"founded": 1975, "employees": 238000, "revenue": "$211.9B"},
}


def lookup_company(name: str) -> str:
    """Look up information about a company."""
    name = name.lower()
    if name in company_data:
        data = company_data[name]
        return f"{name.title()}: Founded {data['founded']}, {data['employees']} employees, {data['revenue']} revenue"
    return f"No data found for {name}"


def calculate_age(founded_year: int) -> int:
    """Calculate how old a company is."""
    from datetime import datetime

    current_year = datetime.now().year
    return current_year - founded_year


# Research assistant with ReAct
researcher = Branch(
    system="You are a research assistant. Look up information and analyze it step by step.",
    tools=[lookup_company, calculate_age],
)

In [6]:
# Research question
question = "Which company is oldest: Apple, Google, or Microsoft? How much older is the oldest than the newest?"

# Let ReAct figure out the steps
research_result = await researcher.ReAct(
    instruct={"instruction": question}, max_extensions=10, verbose=True
)

print("\n📊 Research Result:")
print(research_result)

Invoking action lookup_company with {'name': 'Apple'}.
Action lookup_company invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: To find out which company is oldest among Apple, Google, and Microsoft, and how much older it is compared to the newest, I need to look up the founding years of each company. First, I will check the founding year of Apple, then Google, and finally Microsoft. Once I have all the dates, I can compare them to determine the age difference.
planned_actions:
  - action_type: lookup_company
    description: Determine the founding years of Apple, Google, and Microsoft to identify the oldest and newest companies.
extension_needed: True
milestone:

action_strategy: sequential
action_batch_size:

reason:

action_required: True
action_requests:
  - function: lookup_company
    arguments:
      name: Apple
action_responses:
  - function: lookup_company
    arguments:
      name: Apple
    output: Apple: Founded 1976, 164000 employees, $394.3B revenue
```
---------

Invoking action lookup_company with {'name': 'Google'}.
Action lookup_company invoked, status: completed.


### ReAct Round No.2 Analysis:
```yaml
analysis: We already have Apple founded in 1976. Microsoft was founded earlier in 1975. To determine which is the oldest and calculate the difference, I need Google's founding year. By confirming Google's founding year, I can establish the oldest company among these three and how much older the oldest is compared to the newest.
planned_actions:
  - action_type: lookup_company
    description: Retrieve the founding year of Google to compare with Apple (1976) and Microsoft (1975).
extension_needed: True
milestone:

action_strategy: sequential
action_batch_size:

reason:

action_required: True
action_requests:
  - function: lookup_company
    arguments:
      name: Google
action_responses:
  - function: lookup_company
    arguments:
      name: Google
    output: Google: Founded 1998, 182000 employees, $307.4B revenue
```
---------

Invoking action calculate_age with {'founded_year': 1975}.
Action calculate_age invoked, status: completed.


...

Invoking action calculate_age with {'founded_year': 1975}.
Action calculate_age invoked, status: completed.


### ReAct Round No.4 Analysis:
```yaml
analysis: Microsoft was founded in 1975, making it the oldest among the three companies. Google, founded in 1998, is the newest. To find the difference in age between Microsoft and Google, I need to calculate the number of years between 1975 and 1998.
planned_actions:
  - action_type: lookup_company
    description: Retrieve the founding year of Microsoft to compare with Apple (1976) and Google (1998).
extension_needed: True
milestone:

action_strategy: sequential
action_batch_size:

reason:

action_required: True
action_requests:
  - function: calculate_age
    arguments:
      founded_year: 1975
action_responses:
  - function: calculate_age
    arguments:
      founded_year: 1975
    output: 50
```
---------

### ReAct Round No.5 Analysis:
```yaml
analysis: Microsoft, founded in 1975, is the oldest company among Apple, Google, and Microsoft. Google, founded in 1998, is the newest. The difference in their ages is 23 years (from 1975 to 1998). Since this information is confirmed, no further actions are required.
planned_actions:
  - action_type: calculate_age
    description: Microsoft was founded in 1975, making it the oldest among the three companies. To confirm the age difference between Microsoft (1975) and Google (1998), I will use the stored calculation that shows Microsoft is 50 years old and Google is 25 years old (based on previous calculation).
extension_needed: False
milestone:

action_strategy: sequential
action_batch_size:

reason:

action_required: False
action_requests:
```
---------

### ReAct Final Answer:
The oldest company among Apple, Google, and Microsoft is Microsoft, founded in 1975. It is 23 years older than Google, the newest, founded in 1998.
---------


📊 Research Result:
The oldest company among Apple, Google, and Microsoft is Microsoft, founded in 1975. It is 23 years older than Google, the newest, founded in 1998.


## ReAct vs Regular Function Calling

Let's compare ReAct with regular `operate()` to see the difference:

In [7]:
# Regular function calling approach
regular_branch = Branch(
    system="Answer questions about companies.",
    tools=[lookup_company, calculate_age],
)

# Same question, different approach
regular_result = await regular_branch.operate(
    instruction=question, actions=True
)

print("🔧 Regular operate() result:")
print(f"\nTools used: {len(regular_result.action_responses)}")

print("\n" + "=" * 50)
print(
    "🧠 ReAct approach showed more reasoning steps and systematic problem solving!"
)

🔧 Regular operate() result:

Tools used: 1

🧠 ReAct approach showed more reasoning steps and systematic problem solving!


## When to Use ReAct

**ReAct is best for:**
- Complex, multi-step problems
- When you want to see the reasoning process
- Problems that need strategic tool use
- Research and analysis tasks

**Regular `operate()` is better for:**
- Simple, single-step actions
- When you just want the answer quickly
- Straightforward tool usage

## Simple ReAct Configuration

Key parameters you can control:

In [8]:
# Different ReAct configurations
demo_branch = Branch(
    system="You solve problems step by step.", tools=[add, multiply, divide]
)

# Quick ReAct (fewer steps)
quick_result = await demo_branch.ReAct(
    instruct={"instruction": "What's 10 * 5 + 3?"},
    max_extensions=10,
    verbose=False,  # Hide reasoning steps
)

print("Quick ReAct:", quick_result)

# Detailed ReAct (more steps)
detailed_result = await demo_branch.ReAct(
    instruct={"instruction": "Calculate (15 * 4) + (20 / 5) - 3"},
    max_extensions=10,
    verbose=True,  # Show reasoning steps
)

print("\nDetailed ReAct:", detailed_result)

Quick ReAct: 53
Invoking action multiply with {'a': 15, 'b': 4}.
Action multiply invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: First, multiply 15 by 4 to get 60. Then, divide 20 by 5 to get 4. Next, add 60 and 4 to get 64. Finally, subtract 3 from 64 to reach the result.
planned_actions:
  - action_type: multiply
    description: Calculate 15 multiplied by 4.
extension_needed: False
milestone:

action_strategy: sequential
action_batch_size:

reason:

action_required: True
action_requests:
  - function: multiply
    arguments:
      a: 15
      b: 4
action_responses:
  - function: multiply
    arguments:
      a: 15
      b: 4
    output: 60
```
---------

### ReAct Final Answer:
61
---------


Detailed ReAct: 61


## Key Takeaways

**ReAct Pattern:**
1. 🧠 **Think** about what needs to be done
2. 🔧 **Act** using available tools
3. 🔄 **Repeat** until problem is solved

**Benefits:**
- More reliable problem solving
- Transparent reasoning process
- Better handling of complex tasks
- Strategic tool usage

**Next**: You now understand ReAct fundamentals! This prepares you for more advanced reasoning patterns and complex workflows.